# Issue a VC to the Manufacturer Agents

In [2]:
%%javascript
document.title='Authority Agent'

<IPython.core.display.Javascript object>

#### Imports

In [3]:
# autowait: allows futures and coroutines in the REPL to be awaited
%autoawait 
from aries_cloudcontroller import AriesAgentController
import asyncio
import os
from pprintpp import pprint
from termcolor import colored

IPython autoawait is `on`, and set to use `asyncio`


#### Get schema and cred def identifiers defined in `00_init_authority_as_issuingAuthority.ipynb`

In [4]:
# If the identifiers are not stored for some reason
try:
    # Load variables – assuming they have been written to the store in a previous notebook
    print("--- LOAD VARIABLES FROM JUPYTERLAB STORE ---")
    
    %store -r schema_city_id
    %store -r cred_def_city_id
    %store -r schema_manufacturer_id
    %store -r cred_def_manufacturer_id
    
    print(f"schema_city_id: {schema_city_id}")
    print(f"cred_def_city_id: {cred_def_city_id}")
    print(f"schema_manufacturer_id: {schema_manufacturer_id}")
    print(f"cred_def_manufacturer_id: {cred_def_manufacturer_id}")
    
except Exception as e:
    
    print("--- DEFINE VARIABLES FROM MANUALLY COPIED identifiers DICT ---")
    
    # Copy an updated version if necessary
    identifiers = {
        'city_schema_identifiers': {
            'cred_def': 'BQmsjXtsjzzmTtdwQg5Ftn:3:CL:239757:default',
            'schema_id': 'BQmsjXtsjzzmTtdwQg5Ftn:2:certify-city-agency:0.0.1',
        },
        'manufacturer_schema_identifiers': {
            'cred_def': 'BQmsjXtsjzzmTtdwQg5Ftn:3:CL:239761:default',
            'schema_id': 'BQmsjXtsjzzmTtdwQg5Ftn:2:certify-manufacturer:0.0.1',
        },
    }

    # Get city schema identifiers
    schema_city_id = identifiers["city_schema_identifiers"]["schema_id"]
    cred_def_city_id = identifiers["city_schema_identifiers"]["cred_def"]

    # Get manufacturer schema identifiers
    schema_manufacturer_id = identifiers["manufacturer_schema_identifiers"]["schema_id"]
    cred_def_manufacturer_id = identifiers["manufacturer_schema_identifiers"]["cred_def"]

    # Load variables to Jupyter store
    %store schema_city_id
    %store cred_def_city_id
    %store schema_manufacturer_id
    %store cred_def_manufacturer_id

--- LOAD VARIABLES FROM JUPYTERLAB STORE ---
schema_city_id: BQmsjXtsjzzmTtdwQg5Ftn:2:certify-city-agency:0.0.1
cred_def_city_id: PvCigHE6ZGaR27buqP8Vv1:3:CL:239757:default
schema_manufacturer_id: BQmsjXtsjzzmTtdwQg5Ftn:2:certify-manufacturer:0.0.1
cred_def_manufacturer_id: PvCigHE6ZGaR27buqP8Vv1:3:CL:239761:default


#### Init agent controller of Authority

In [5]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://authority-agent:3021 and an api key of adminApiKey


## 1 – Init Webhook Server and register Event Listeners for it
#### 1.1 – Start webhook server
Start a webhook server to be able to communicate with other agents

In [6]:
# Setup
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.2 – Define listeners
Define listeners that are triggered when something happens on the webhook server

In [7]:
# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : ", routing_state)
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")

    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "request":
        # Your business logic
        print("request")

    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))


def issuer_handler(payload):
    connection_id = payload['connection_id']
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    print("\n---------------------------------------------------\n")
    print("Handle Issue Credential Webhook")
    print(f"Connection ID : {connection_id}")
    print(f"Credential exchange ID : {exchange_id}")
    print("Agent Protocol Role : ", role)
    print("Protocol State : ", state )
    print("\n---------------------------------------------------\n")
    
    
    if state == "offer_sent":
        proposal = payload["credential_proposal_dict"]
        attributes = proposal['credential_proposal']['attributes']
        print(f"Offering : \n {attributes}")
        ## YOUR LOGIC HERE
    elif state == "request_received":
        print("Request for credential received")
        ## YOUR LOGIC HERE
    elif state == "credential_sent":
        print("Credential Sent")
        ## YOUR LOGIC HERE
        
        
def messages_handler(payload):
    connection_id = payload["connection_id"]
    print("Handle message", connection_id)
    pprint(payload)

In [8]:
# Init listeners list
listeners = []
        
# Add listeners defined in previous cell
connection_listener = {"handler": connections_handler, "topic": "connections"}
listeners.append(connection_listener)

issuer_listener = {"handler": issuer_handler, "topic": "issue_credential"}
listeners.append(issuer_listener)

message_listener = {"handler": messages_handler, "topic": "basicmessages"}
listeners.append(message_listener)

#### 1.3 – Register listeners with `agent_controller`

In [9]:
#loop = asyncio.get_event_loop()
#loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners(listeners)

**BREAK POINT:** Please switch to agent `Manufacturer1`, open `01_hold_VC.ipynb`, and start executing the notebook

---

## 2 – Establish connection with `Manufacturer1` agent
Establish connection with the`Manufacturer1` agent. In this scenario, the `Manufacturer1` sends an invitation to connect with the Authority. 

### 2.1 – Receive invitation from `Manufacturer1` agent
Copy the invitation from Step 2.2 in the City's `01_hold_VC.ipynb` notebook into the following cell. 

In [10]:
invitation = {
    '@id': '96858149-90aa-430a-9413-f040d0f88da9',
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
    'label': 'Manufacturer1',
    'recipientKeys': ['2YKxmwVqURYm7rUqrHX5M5F33fWGHgJvjXBMrTP7Neh4'],
    'serviceEndpoint': 'http://510450e76bb6.ngrok.io',
}

# Receive invitation and store the connection_id for further protocols (e.g., issuing VCs)
invite_response = await agent_controller.connections.receive_invitation(invitation, alias=None, auto_accept=False)
connection_id = invite_response["connection_id"]

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  5d80c45b-e7f7-4834-95d2-16f41d44bf53
State :  invitation
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
invitation


### 2.2 – Accept invitation

In [11]:
# Label for the connection
my_label = "City Agent <> Authority Agent" # Label for connection
my_endpoint = None # Endpoint you expect to recieve messages at

# Accept response
accept_response = await agent_controller.connections.accept_invitation(connection_id, my_label, my_endpoint)

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  5d80c45b-e7f7-4834-95d2-16f41d44bf53
State :  request
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
request
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  5d80c45b-e7f7-4834-95d2-16f41d44bf53
State :  response
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
response
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  5d80c45b-e7f7-4834-95d2-16f41d44bf53
State :  active
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
Connection ID: 5d80c45b-e7f7-4834-95d2-16f41d44bf53 is now active.
Handle message 5d80c45b-e7f7-4834-95d2-16f41d44bf53
{
    'connection_id': '5d80c45b-e7f7-4834-95d2-16f41d44bf53',


**BREAK POINT:** Please proceed to step 2.3 in the `01_hold_VC.ipynb` notebook of the `Manufacturer1` agent

---

## 3 – Get info from `Manufacturer1` agent and issue a VC
### 3.1 – Greet `Manufacturer1` to open connection in this cell
The response with the relevant City information will be displayed in the following cell.

In [12]:
basic_message = "Hello Manufacturer1. Plase provide me with the following information: manufacturerName, manufacturerCountry, ManufacturerCity."
await agent_controller.messaging.send_message(connection_id, basic_message)

{}

Handle message 5d80c45b-e7f7-4834-95d2-16f41d44bf53
{
    'connection_id': '5d80c45b-e7f7-4834-95d2-16f41d44bf53',
    'content': 'Manufacturer1 received your message',
    'message_id': 'ff2cf16c-3b79-4234-8d8a-c037372d4731',
    'sent_time': '2021-08-05 07:43:01.690154Z',
    'state': 'received',
}
Handle message 5d80c45b-e7f7-4834-95d2-16f41d44bf53
{
    'connection_id': '5d80c45b-e7f7-4834-95d2-16f41d44bf53',
    'content': '{"manufacturerName": "M1", "manufacturerCountry": "DE", "manufacturerCity": "Munich"}',
    'message_id': 'a7d2bdcc-d3ec-49fc-baaa-69404c7fcfe4',
    'sent_time': '2021-08-05 07:43:07.675398Z',
    'state': 'received',
}


### 3.2 – Populate VC Attributes

Define the values that will be issued in the VC. The attributes match the attributes defined in the city-agent-scheme.

In [13]:
values_m1 = {"manufacturerName": "M1", "manufacturerCountry": "DE", "manufacturerCity": "Munich"}
credential_attributes = [
    {"name": "manufacturerName", "value": values_m1["manufacturerName"]},
    {"name": "manufacturerCity", "value": values_m1["manufacturerCity"]},
    {"name": "manufacturerCountry", "value": values_m1["manufacturerCountry"]},
    {"name": "isManufacturer", "value": "TRUE"}
]
print(credential_attributes)

[{'name': 'manufacturerName', 'value': 'M1'}, {'name': 'manufacturerCity', 'value': 'Munich'}, {'name': 'manufacturerCountry', 'value': 'DE'}, {'name': 'isManufacturer', 'value': 'TRUE'}]


### 3.3 – Offer VC to `Manufacturer1` agent

In [14]:
# MAKE VC ZKP-able! SEE https://github.com/hyperledger/aries-cloudagent-python/blob/main/JsonLdCredentials.md
trace = False # Don't trace ACA-PY instance
comment = "Issuing VC that Manufacturer1 is a manufacturer"
auto_remove = True # Remove credential record after issued?

# Cred_def_id must identify a definition to which your agent has corresponding private issuing key.
send_cred_response = await agent_controller.issuer.send_credential(connection_id, schema_manufacturer_id, cred_def_manufacturer_id, credential_attributes, comment, auto_remove, trace)


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 5d80c45b-e7f7-4834-95d2-16f41d44bf53
Credential exchange ID : d3200cd0-ec61-4ae3-8b9e-588a83d7f94d
Agent Protocol Role :  issuer
Protocol State :  offer_sent

---------------------------------------------------

Offering : 
 [{'name': 'manufacturerName', 'value': 'M1'}, {'name': 'manufacturerCity', 'value': 'Munich'}, {'name': 'manufacturerCountry', 'value': 'DE'}, {'name': 'isManufacturer', 'value': 'TRUE'}]

---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 5d80c45b-e7f7-4834-95d2-16f41d44bf53
Credential exchange ID : d3200cd0-ec61-4ae3-8b9e-588a83d7f94d
Agent Protocol Role :  issuer
Protocol State :  request_received

---------------------------------------------------

Request for credential received

---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 5d80c45b-e7f7-4834-95d2-16f41d4

**BREAK POINT:** Proceed to `Manufacturer1` Agent, Step 3.2 

---

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [15]:
await agent_controller.terminate()